In [86]:
%matplotlib inline

In [45]:
import argparse
import logging
import os
import os.path as osp
import sys
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
# Change this path to wherever you installed the Pytorch-UNet module
sys.path.append(r"C:\Users\Groh\Documents\GitHub\unet-nested-multiple-classification")
# sys.path.append(r"C:\Users\Groh\Documents\GitHub\colabtools\google")
os.environ['KMP_DUPLICATE_LIB_OK']='True'

import cv2
import numpy as np
import torch
from torch.utils.data import DataLoader, Dataset, random_split
import torch.nn.functional as F
from torchvision.transforms import functional as func
from torchvision import transforms
from PIL import Image
from tqdm import tqdm

from unet import NestedUNet
from unet import UNet
from utils.dataset import BasicDataset
from config import UNetConfig

# from google.colab import drive
# drive.mount('/content/drive')
# os.chdir("/content/drive/My Drive")

In [34]:
# Evaluation function
def eval_net(net, loader, device, n_val, cfg):
    """
    Evaluation without the densecrf with the dice coefficient

    """
    net.eval()
    tot = 0

    with tqdm(total=n_val, desc='Validation round', unit='img', leave=False) as pbar:
        for batch in loader:
            imgs = batch['image']
            true_masks = batch['mask']

            imgs = imgs.to(device=device, dtype=torch.float32)
            mask_type = torch.float32 if cfg.n_classes == 1 else torch.long
            true_masks = true_masks.to(device=device, dtype=mask_type)

            # compute loss
            if cfg.deepsupervision:
                masks_preds = net(imgs)
                loss = 0
                for masks_pred in masks_preds:
                    print(masks_pred.shape)
                    tot_cross_entropy = 0
                    for true_mask, pred in zip(true_masks, masks_pred):
                        pred = (pred > cfg.out_threshold).float()
                        if cfg.n_classes > 1:
                            sub_cross_entropy = F.cross_entropy(pred.unsqueeze(dim=0), true_mask.unsqueeze(dim=0).squeeze(1)).item()
                        else:
                            sub_cross_entropy = dice_coeff(pred, true_mask.squeeze(dim=1)).item()
                        tot_cross_entropy += sub_cross_entropy
                    tot_cross_entropy = tot_cross_entropy / len(masks_preds)
                    tot += tot_cross_entropy
            else:
                masks_pred = net(imgs)
                for true_mask, pred in zip(true_masks, masks_pred):
                    pred = (pred > cfg.out_threshold).float()
                    if cfg.n_classes > 1:
                        tot += F.cross_entropy(pred.unsqueeze(dim=0), true_mask.unsqueeze(dim=0).squeeze(1)).item()
                    else:
                        tot += dice_coeff(pred, true_mask.squeeze(dim=1)).item()

            pbar.update(imgs.shape[0])

    return tot / n_val


In [3]:
cfg = UNetConfig()

def inference_one(net, image, device):
    net.eval()

    img = torch.from_numpy(BasicDataset.preprocess(image, cfg.scale))

    img = img.unsqueeze(0)
    img = img.to(device=device, dtype=torch.float32)

    with torch.no_grad():
        output = net(img)
        if cfg.deepsupervision:
            output = output[-1]

        if cfg.n_classes > 1:
            probs = F.softmax(output, dim=1)
        else:
            probs = torch.sigmoid(output)

        probs = probs.squeeze(0)

        tf = transforms.Compose(
            [
                transforms.ToPILImage(),
                transforms.Resize((image.size[1], image.size[0])),
                transforms.ToTensor()
            ]
        )
        
        if cfg.n_classes == 1:
            probs = tf(probs.cpu())
            mask = probs.squeeze().cpu().numpy()
            return mask > cfg.out_threshold
        else:
            masks = []
            for prob in probs:
                prob = tf(prob.cpu())
                mask = prob.squeeze().cpu().numpy()
                mask = mask > cfg.out_threshold
                masks.append(mask)
            return masks

In [16]:
all_images = r"C:\Users\Groh\Desktop\TrainingDigEndo\images_all"
sessions_to_test = ["09","10"]

In [14]:
ttt = ["09","10"]
print(tuple(["aicm"+i for i in ttt]))

('aicm09', 'aicm10')


In [22]:
def getSelectedFilenames(directory, sessions):
  all_filenames = list(sorted(os.listdir(directory)));
  #print(all_filenames)
  selected_filenames = []
  
  for session in sessions:
    string = 'aicm' + session;
    for name in all_filenames:
      if string in name:
       selected_filenames.append(name);

  #print(selected_filenames)   
  return selected_filenames;

#getSelectedFilenames(dir_img, ['05_02', '06']);

In [24]:
dir_img = r"C:\Users\Groh\Desktop\TrainingDigEndo\images_training"
dir_mask = r"C:\Users\Groh\Desktop\TrainingDigEndo\masks_grayscale"

In [28]:
class CustomDataset(Dataset):
    def __init__(self, images_directory, masks_directory, sessions, transform=None):   
        self.images_filenames = getSelectedFilenames(images_directory, sessions);
        self.images_directory = images_directory;
        self.masks_directory = masks_directory;
        self.transform = transform;
        self.sessions = sessions  

        #print(self.images_filenames[0])             

    def __len__(self):
        return len(self.images_filenames)

    def __getitem__(self, idx):
        #print(self.images_filenames)
        image_filename = self.images_filenames[idx]

        pathImage = os.path.join(self.images_directory, image_filename);
        pathMask = os.path.join(self.masks_directory, image_filename);
        
        pillow_image = Image.open(pathImage)
        image = np.array(pillow_image)
        pillow_mask = Image.open(pathMask)
        mask = np.array(pillow_mask)
        
        if self.transform is not None:
          transformed = self.transform(image=image, mask=mask)
          image = transformed["image"]
          mask = transformed["mask"]
          image = np.array(image)
          mask = np.array(mask)
         
          #to get channels_first format
          #image_np = moveaxis(image_np, 2,0)
                    
         
          #image_tensor = torch.from_numpy(image_np).type(torch.FloatTensor)
          #mask_tensor = torch.from_numpy(mask_np).type(torch.FloatTensor)

        image = func.to_tensor(np.array(image))
        mask = func.to_tensor(np.array(mask)) 


        return {
            'image': image,
            'mask':  mask
        } 

    def preprocess(cls, pil_img, scale):
        w, h = pil_img.size
        newW, newH = int(scale * w), int(scale * h)
        assert newW > 0 and newH > 0, 'Scale is too small'
        pil_img = pil_img.resize((newW, newH))

        img_nd = np.array(pil_img)

        if len(img_nd.shape) == 2:
            img_nd = np.expand_dims(img_nd, axis=2)

        # HWC to CHW
        img_trans = img_nd.transpose((2, 0, 1))
        if img_trans.max() > 1:
            img_trans = img_trans / 255

        return img_trans         
        
val_dataset = CustomDataset(dir_img, dir_mask, ["09","10"])

In [35]:
n_val = len(val_dataset)

val = val_dataset

val_loader = DataLoader(val,
                        batch_size=cfg.batch_size,
                        shuffle=False,
                        num_workers=0,
                        pin_memory=True)


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

val_score = eval_net(net, val_loader, device, n_val, cfg)

print(val_score)

# net = eval(cfg.model)(cfg)
# test_img = 
# masks_preds = net(imgs)

Validation round:   1%|▌                                                              | 1/123 [00:01<03:45,  1.85s/img]

torch.Size([1, 6, 432, 768])
torch.Size([1, 6, 432, 768])
torch.Size([1, 6, 432, 768])
torch.Size([1, 6, 432, 768])


Validation round:   2%|█                                                              | 2/123 [00:03<03:52,  1.92s/img]

torch.Size([1, 6, 432, 768])
torch.Size([1, 6, 432, 768])
torch.Size([1, 6, 432, 768])
torch.Size([1, 6, 432, 768])


Validation round:   2%|█▌                                                             | 3/123 [00:05<03:51,  1.93s/img]

...

Validation round:   4%|██▌                                                            | 5/123 [00:09<03:48,  1.94s/img]

torch.Size([1, 6, 432, 768])
torch.Size([1, 6, 432, 768])
torch.Size([1, 6, 432, 768])
torch.Size([1, 6, 432, 768])


Validation round:   5%|███                                                            | 6/123 [00:11<03:48,  1.95s/img]

torch.Size([1, 6, 432, 768])
torch.Size([1, 6, 432, 768])
torch.Size([1, 6, 432, 768])
torch.Size([1, 6, 432, 768])


KeyboardInterrupt: 

In [104]:
def variability(mask_predic, mask_true):
        # Check whether both images have the same input shape
        if mask_predic.shape != mask_true.shape:
            raise ValueError("The two masks have different shapes. Please adjust the shapes accordingly.")
        else:
            
            intersection=0
            union=0
            
            # Calculate the intersection and the union of both masks
            for i in range(mask_predic.shape[0]):
                for j in range(mask_true.shape[1]):
                    if mask_predic[i][j] and mask_true[i][j]:
                        intersection+=1
                    if mask_predic[i][j] or mask_true[i][j]:
                        union+=1       
            
            # Return dice score, as well as intersection over union (iou)
            # Quality metrics don't make sense, when the given class is not visible, hence the NaNs.
            if not mask_true.any():
                dice = np.NaN
                iou = np.NaN
            else:
                dice = (2. * intersection)/(mask_predic.sum() + mask_true.sum())
                iou = intersection/union
        return dice, iou

In [117]:
sessions_to_test = ["10"]

output_dir = r"C:\Users\Groh\Desktop\Test_Results"
true_masks = r"C:\Users\Groh\Desktop\TrainingDigEndo\masks_grayscale"
checkpoint = r"C:\Users\Groh\Documents\GitHub\unet-nested-multiple-classification\data\checkpoints\epoch_2.pth"

input_imgs = [i for i in os.listdir(dir_img) if os.path.isfile(os.path.join(dir_img,i)) and \
     i.startswith(tuple(["aicm"+ses for ses in sessions_to_test]))]

net = eval(cfg.model)(cfg)
logging.info("Loading model {}".format(checkpoint))

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
logging.info(f'Using device {device}')
net.to(device=device)
net.load_state_dict(torch.load(checkpoint, map_location=device))

logging.info("Model loaded !")

# Preallocate arrays
total_dice = np.zeros((cfg.n_classes, int(len(input_imgs))))
total_dice[:] = np.NaN                  
total_iou = np.zeros((cfg.n_classes, int(len(input_imgs))))
total_iou[:] = np.NaN     

img_count = 0
for i, img_name in tqdm(enumerate(input_imgs)):
    logging.info("\nPredicting image {} ...".format(img_name))

    img_path = osp.join(dir_img, img_name)
    img = Image.open(img_path)

    mask = inference_one(net=net,
                         image=img,
                         device=device)
    for idx in range(0,len(mask)):
        img = cv2.imread(os.path.join(dir_mask, img_name), cv2.IMREAD_GRAYSCALE)
        mask_true = img == idx
        mask_predic = mask[idx]
        dice, iou = variability(mask_predic, mask_true)
        total_dice[idx,img_count] = dice
        total_iou[idx,img_count] = iou
        print("Image {}, Class {} --- Dice Score: {:.5f} | IOU: {:.5f}".format(img_name, idx, dice, iou))
    img_count += 1

for idx in range(0,cfg.n_classes):
    print("\nAverage Dice Score for Class {}: {:.4f} (STD: {:.4f})".format(idx, np.nanmean(total_dice[idx]), np.nanstd(total_dice[idx])))
    print("Average IOU for Class {}: {:.4f} (STD: {:.4f})".format(idx, np.nanmean(total_iou[idx]), np.nanstd(total_iou[idx])))
        

0it [00:00, ?it/s]

Image aicm10_02_000000.png, Class 0 --- Dice Score: 0.00198 | IOU: 0.00099
Image aicm10_02_000000.png, Class 1 --- Dice Score: nan | IOU: nan
Image aicm10_02_000000.png, Class 2 --- Dice Score: 0.03189 | IOU: 0.01620
Image aicm10_02_000000.png, Class 3 --- Dice Score: nan | IOU: nan


1it [00:02,  2.93s/it]

Image aicm10_02_000000.png, Class 4 --- Dice Score: nan | IOU: nan
Image aicm10_02_000000.png, Class 5 --- Dice Score: nan | IOU: nan
Image aicm10_02_000001.png, Class 0 --- Dice Score: 0.00473 | IOU: 0.00237
Image aicm10_02_000001.png, Class 1 --- Dice Score: nan | IOU: nan
Image aicm10_02_000001.png, Class 2 --- Dice Score: 0.03803 | IOU: 0.01938
Image aicm10_02_000001.png, Class 3 --- Dice Score: nan | IOU: nan


2it [00:05,  2.96s/it]

...

36it [01:49,  3.08s/it]

Image aicm10_03_000054.png, Class 4 --- Dice Score: nan | IOU: nan
Image aicm10_03_000054.png, Class 5 --- Dice Score: nan | IOU: nan
Image aicm10_03_000063.png, Class 0 --- Dice Score: 0.02374 | IOU: 0.01201
Image aicm10_03_000063.png, Class 1 --- Dice Score: nan | IOU: nan
Image aicm10_03_000063.png, Class 2 --- Dice Score: nan | IOU: nan
Image aicm10_03_000063.png, Class 3 --- Dice Score: nan | IOU: nan


37it [01:52,  3.03s/it]

Image aicm10_03_000063.png, Class 4 --- Dice Score: nan | IOU: nan
Image aicm10_03_000063.png, Class 5 --- Dice Score: nan | IOU: nan

Average Dice Score for Class 0: 0.0068 (STD: 0.0071)
Average IOU for Class 0: 0.0034 (STD: 0.0036)

Average Dice Score for Class 1: 0.0147 (STD: 0.0090)
Average IOU for Class 1: 0.0074 (STD: 0.0046)

Average Dice Score for Class 2: 0.0199 (STD: 0.0311)
Average IOU for Class 2: 0.0103 (STD: 0.0165)

Average Dice Score for Class 3: nan (STD: nan)
Average IOU for Class 3: nan (STD: nan)

Average Dice Score for Class 4: nan (STD: nan)
Average IOU for Class 4: nan (STD: nan)

Average Dice Score for Class 5: nan (STD: nan)
Average IOU for Class 5: nan (STD: nan)



c:\users\groh\anaconda3\envs\labelme\lib\site-packages\ipykernel_launcher.py:47: RuntimeWarning: Mean of empty slice
c:\users\groh\anaconda3\envs\labelme\lib\site-packages\ipykernel_launcher.py:48: RuntimeWarning: Mean of empty slice


In [36]:
sessions_to_test = ["09"]

output_dir = r"C:\Users\Groh\Desktop\Test_Results"
checkpoint = r"C:\Users\Groh\Documents\GitHub\unet-nested-multiple-classification\data\checkpoints\epoch_2.pth"

input_imgs = [i for i in os.listdir(all_images) if os.path.isfile(os.path.join(all_images,i)) and \
     i.startswith(tuple(["aicm"+ses for ses in sessions_to_test]))]

net = eval(cfg.model)(cfg)
logging.info("Loading model {}".format(checkpoint))

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
logging.info(f'Using device {device}')
net.to(device=device)
net.load_state_dict(torch.load(checkpoint, map_location=device))

logging.info("Model loaded !")

for i, img_name in tqdm(enumerate(input_imgs)):
    logging.info("\nPredicting image {} ...".format(img_name))

    img_path = osp.join(all_images, img_name)
    img = Image.open(img_path)

    mask = inference_one(net=net,
                         image=img,
                         device=device)
    img_name_no_ext = osp.splitext(img_name)[0]
    output_img_dir = osp.join(output_dir, img_name_no_ext)
    os.makedirs(output_img_dir, exist_ok=True)

    if cfg.n_classes == 1:
        image_idx = Image.fromarray((mask * 255).astype(np.uint8))
        image_idx.save(osp.join(output_img_dir, img_name))
    else:
        for idx in range(0, len(mask)):
            img_name_idx = img_name_no_ext + "_" + str(idx) + ".png"
            image_idx = Image.fromarray((mask[idx] * 255).astype(np.uint8))
            image_idx.save(osp.join(output_img_dir, img_name_idx))


116it [04:13,  2.19s/it]


KeyboardInterrupt: 